In [35]:
from estnltk import Text
import json
import os
from collections import defaultdict
import string 
import nltk
from collections import Counter
import math
import re

In [122]:
def faili_info_salvestamine(filename):
    global keskmised
    
    # Loob failinimed
    ga_name = os.path.join("etnc19_web_2019_morf_oletamisega/", ".".join(filename.split(".")[:-1]) + "_morf_oletamisega.json")
    ta_name = os.path.join("etnc19_web_2019_morf_oletamiseta/", ".".join(filename.split(".")[:-1]) + "_morf_oletamiseta.json")
    meta_name = os.path.join("etnc19_web_2019_meta/", ".".join(filename.split(".")[:-1]) + "_meta.json")
    
    andmed_name = os.path.join("etnc19_web_2019_andmed/", ".".join(filename.split(".")[:-1]) + "_meta.json")
    
    # Loeb failidest sisse tekstid ja metainfo
    oletamisega = None
    oletamiseta = None
    meta_info = None
    with open(ga_name, "r", encoding = "utf-8") as fr:
        oletamisega = Text(json.load(fr))
    with open(ta_name, "r", encoding = "utf-8") as fr:
        oletamiseta = Text(json.load(fr))
    with open(meta_name, "r", encoding = "utf-8") as fr:
        meta_info = json.load(fr)
    
    # Väljastatavate andmete sõnastik
    andmed = defaultdict(float)
    
    # Metainfost oluliste andmete meelde jätmine
    andmed["emotikonide_arv"] = meta_info["emotikonide_arv"]
    
    # Keskmistest varieerimise arvutamine
    andmed["TTR_varieerumine"] = meta_info["TTR"] - keskmised['TTR']
    andmed["käändsõnade_varieerumine"] = meta_info["käänduvate_lemmade_osaarv"] - keskmised['käänduvate_lemmade_osaarv']
    andmed["lemmapikkuse_varieerumine"] = meta_info["keskmine_lemma_pikkus"] - keskmised['keskmine_lemma_pikkus']
    
    # Arvutab tajuverbide osaarvu kõikidest verbidest ja jätab meelde
    andmed['tajuverbide_osaarv'] = tajuverbide_keskmine(oletamisega)
    
    # Arvutab lühikeste tundmatu morfanalüüsi saanud sõnade osaarvu
    andmed['luhemate_tundmatute_osakaal'] = luhemate_tundmatute_osakaal(oletamiseta)
    
    # Loendab kokku vähemalt kolm korda korduvad tähed sõna sees
    andmed['korduvate_tähtede_arv'] = leia_korduvad_tähed(oletamisega)
    
    # Leiab verbide isikute protsendid (kui palju on 1., 2. ja 3. isik)
    verbide_isikute_protsendid = verbide_isikute_osakaalud(oletamisega)
    andmed['verbide_esimese_isiku_osaarv'] = verbide_isikute_protsendid[0]
    andmed['verbide_teise_isiku_osaarv'] = verbide_isikute_protsendid[1]
    andmed['verbide_kolmanda_isiku_osaarv'] = verbide_isikute_protsendid[2]
    
    # Leiab asesõnade isikute protsendid (kui palju on 1., 2. ja 3. isik)
    asesonade_isikute_protsendid = asesonade_isikute_osakaalud(oletamisega)
    andmed['asesõnade_esimese_isiku_osaarv'] = asesonade_isikute_protsendid[0]
    andmed['asesõnade_teise_isiku_osaarv'] = asesonade_isikute_protsendid[1]
    andmed['asesõnade_kolmanda_isiku_osaarv'] = asesonade_isikute_protsendid[2]
    
    # Leiab passiivi osakaalu ja jätab meelde
    andmed['passiivi_osakaal'] = passiivi_osakaal(oletamisega)
    
    # Kirjutab metainfo tagasi faili edasiseks skoori arvutamiseks
    with open(andmed_name, 'w', encoding="UTF-8") as fw:
        json.dump(andmed, fw, sort_keys=True, indent=4)

In [10]:
#Tajuverbide keskmise arvutamine
def tajuverbide_keskmine(oletamisega):
    with open("Loendid\\tajuverbid\wordnet_tajuverbid.txt", "r", encoding = "utf8") as fr:
        lines = fr.readlines()
        tajuverbid = [verb.strip() for verb in lines]
        
    all_verbs = 0
    only_tajuverbs = 0

    for lemma, postag in zip(oletamisega.lemmas, oletamisega.postags):
        if postag == "V":
            all_verbs += 1
            if lemma in tajuverbid:
                only_tajuverbs += 1

    return only_tajuverbs / float(all_verbs)

In [53]:
#Korduvate tähtede leidmine
def leia_korduvad_tähed(oletamisega):
    rx = re.compile(r'(\D)\1{2,}')
    rxx = rx.findall(oletamisega.text)
    return len(rxx)

In [31]:
#Tundmatude sõnade osakaalu leidmine
def luhemate_tundmatute_osakaal(oletamiseta):
    analuusita = 0
    koik = 0

    vordlus = int(math.ceil(keskmised["keskmine_lemma_pikkus"]*2))

    for word in oletamiseta.words:
        if word["analysis"] == []:
            if not all(char in string.punctuation for char in word["text"]):
                if len(word["text"]) < vordlus:
                    analuusita += 1
                    
    return analuusita / len(oletamiseta.words)

In [119]:
#Esimese, teise ja kolmanda isiku osakaalu leidmine verbidest
def verbide_isikute_osakaalud(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    # Ei kordu omavahel
    esi_tunnused = ['sime', 'me', 'nuksime', 'nuksin', 'gem', 'ksin', 'n', 'sin', 'ksime']
    teine_tunnused = ['te', 'o', 'nuksite', 'd', 'site', 'ge', 'ksite']
    kolmas_tunnused = ['s', 'gu', 'vad', 'b']
    
    # Loeb kokku isikute kaupa ja kõik isikud kokku
    arv_koik = 0
    arv_esimene = 0
    arv_teine = 0
    arv_kolmas = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis) == 1:
            analysis = analysis[0]
            # Kui tegu on verbiga
            if analysis['partofspeech'] == "V":
                #Jätab meelde sõnalõpu
                form = analysis['form']
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in esi_tunnused:
                    arv_koik += 1
                    arv_esimene += 1
                    continue
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                elif form in teine_tunnused:
                    arv_koik += 1
                    arv_teine += 1
                    continue
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                elif form in kolmas_tunnused:
                    arv_koik += 1
                    arv_kolmas += 1
                    continue
    # Tagastab kõik kolm osaarvu
    return arv_esimene/arv_koik, arv_teine/arv_koik, arv_kolmas/arv_koik

In [116]:
#Esimese, teise ja kolmanda isiku osakaalu leidmine asesõnadest
def aesõnade_isikute_osakaalud(oletamisega):
    # Loeb kokku isikute kaupa ja kõik isikud kokku
    arv_koik = 0
    arv_esimene = 0
    arv_teine = 0
    arv_kolmas = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis) == 1:
            analysis = analysis[0]
            # Kui tegu on asesõnaga
            if analysis['partofspeech'] == "P":
                #Jätab meelde asesõna
                lemma = analysis['lemma']
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                if lemma == 'mina':
                    arv_koik += 1
                    arv_esimene += 1
                    continue
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                #Teietamine automaatselt sama, mis sinatamine (sina mitmuse analüüs)
                elif lemma == 'sina':
                    arv_koik += 1
                    arv_teine += 1
                    continue
                #Vaatab asesõna algvormi ja suurendab vastavat skoori
                elif lemma == 'tema':
                    arv_koik += 1
                    arv_kolmas += 1
                    continue
    # Tagastab kõik kolm osaarvu
    return arv_esimene/arv_koik, arv_teine/arv_koik, arv_kolmas/arv_koik

In [123]:
#Umbisikuliste verbide osakaalu leidmine tekstist
def passiivi_osakaal(oletamisega):
    # Tunnuste lõpud, EstNLTK dokumentatsioonist
    tunnused = ['takse', 'ti', 'tav', 'tuks', 'tagu', 'tama', 'tud', 'tuvat', 'tavat', 'taks', 'ta']
    
    # Loeb kokku isikute kaupa ja kõik isikud kokku
    arv_koik = 0
    arv_passiiv = 0
    
    # Vaatab iga sõna analüüsi
    for analysis in oletamisega.analysis:
        # Kui pole mitmese analüüsiga
        if len(analysis) == 1:
            analysis = analysis[0]
            # Kui tegu on verbiga
            if analysis['partofspeech'] == "V":
                #Jätab meelde sõnalõpu
                form = analysis['form']
                # Vaatab, kas tunnus on loendis, ja suurendab vastavalt skoori
                if form in tunnused:
                    arv_koik += 1
                    arv_passiiv += 1
                    continue
                else:
                    arv_koik += 1
        # Kui on mitmese analüüsiga
        else:
            leidub_passiivne = False
            # Vaatab kõiki analüüse
            # Kui vähemalt üks on passiivi analüüsiga
            # Märgib kogu leiduva vormi passiivseks
            for analuus in analysis:
                if analuus['partofspeech'] == "V":
                    # Vaatab, kas tunnus on loendis, ja märgib, et järelikult on passiiviga
                    if analuus['form'] in tunnused:
                        leidub_passiivne = True
                        break
            
            if leidub_passiivne:
                arv_koik += 1
                arv_passiiv += 1
            else:
                arv_koik += 1
        
    # Tagastab osaarvu
    return arv_passiiv/arv_koik

In [114]:
# Algse kausta nimi, võtab sealt algsed failinimed, ei loe neid faile sisse
source = "etnc19_web_2019_100000/"

os.makedirs(os.path.dirname("etnc19_web_2019_andmed/"), exist_ok=True)

with open("keskmised.json", "r", encoding = "UTF-8") as fr:
    keskmised = json.load(fr)
    
# Avab järjest kõik failid algkaustas
for file in [f for f in os.listdir(source)]:
    # Arvutab faili kohta info
    faili_info_salvestamine(file)

KeyboardInterrupt: 

Katsetamine

In [102]:
with open("etnc19_web_2019_morf_oletamiseta\www.etwinning.net_4188741_morf_oletamiseta.json", "r", encoding = "utf-8") as fr:
    oletamiseta = Text(json.load(fr))
with open("etnc19_web_2019_morf_oletamisega\www.1helin.wordpress.com_4952900_morf_oletamisega.json", "r", encoding = "utf-8") as fr:
    oletamisega = Text(json.load(fr))

In [124]:
passiivi_osakaal(oletamisega)

0.043478260869565216